# Convolutional neural networks

## Why?

- Fully Connected networks look at input vector as an independent features
- Images are __clearly__ dependent; one can estimate pixel values from the ones surrounding it (or vice versa)
- Convolution takes this interaction an account (relative position of feature is used)
- Convolution models spatial relationships between features (not only images, but also soundwaves, 3D models, videos, words in sentences etc.)
- Smaller number of parameters allows us to use deeper and more complicated layers
- Simpler architecture (less parameters), tailored to spatially structured data
- __Convolutional neural networks find higher level features (a.k.a. representations) useful for final classification layers__

## What is convolution?

> Kernel (vector/matrix/cube) "sliding" over input data (multiplying it with kernel values) and summing them together

For binary case:

![image](images/convolution_animation.gif)

- Kernel is applied on first image patch. It's values are multiplied by respective values in the image
- After multiplication all of those values are summed __returning single element__ (you can think of it as a new pixel.
- This process is repeated until end of image is reached and new matrix is created.

Below are change'able arguments of convolution (those are it's __hyperparameters__):

# Convolution parameters

## Kernel size

> Dimensionality of kernel. In case above it was `3x3` kernel (pretty popular choice).

- It can be specified as a tuple, e.g. `(3, 3)`
- It can be irregular (e.g. `(3, 2)`), __though it is rarely a case__ (if it is, it almost always is `(N, 1)` or `(1, N)`)
- The larger the kernel, the larger is it's __receptive field__ but more computations have to be performed

## Stride

> Number of pixels we shift our kernel in a certain direction. In case above it was `(1, 1)`

- It can be specified as a tuple, e.g. `(1, 1)`
- It can be irregular (e.g. `(2, 1)`), __though it is almost never a case__ (specific use cases, __we shouldn't be concerned with this possibility__
- The larger the stride, the more features from original image we miss
- The larger the stride, the smaller output image becomes
- The larger the stride, the less operations have to be performed
- __Due to above `stride=1` is the most common value__


## Padding

> Addition values around the image (usually zeros). In case above __there was no padding (a.k.a. "valid" padding)__

![image](images/CNN_diagram.JPG)

- It can be specified as a tuple, e.g. `(1, 1)`
- One can choose from a few modes, specifically (see [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)):
    - `zeros` - output is padded with zero values; __most common & default__
    - `reflect` - output is reflected, same as `replicate` for `1`, after that consecutive neighboring values ared used; __used for medical imaging, sometimes image segmentation etc.__
    - `replicate` - last value at the border is used; __rare__
    - `circular` - first and consecutive values are used; __rare__

### Okay but why?

> Without padding image shrinks (when `kernel_size` > 1)

Assume we stack a few convolutions, one after another. __After a while our image will be a single pixel!__

- If we add padding we can preserve image's size without introducing noise (or at least too much of it)
- Pixels on the edge do not contribute as much to the kernel. If we add appropriate padding they contribute the same as the rest.
- Acts as a mild regularizer (depending on the mode)

### Tips

- Specifying `padding` with "default values" (like `stride=1`, `dilation=1`) is easy and can be done using:

$$
\lfloor\frac{\text{kernel_size}}{2}\rfloor
$$

- Exact output size dependent on parameters is provided by PyTorch in [Shape](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
- Some libraries provide padding `"same"` which calculates padding for you dynamically ([torchlayers](https://github.com/szymonmaszke/torchlayers) for PyTorch, Keras & Tensorflow have it out of the box)

## Dilation

> Dilation means spacing between kernel elements

![dilated](images/dilated.jpg)

- Increased receptive field
- Some pixels are missed but this information can be approximated
- __Also called `atrous convolution`__

### Tips

- Useful for very large images
- Useful when we want to get more global representations of images
- Most useful in the early layers
- Can be concatenated with standard convolution

# Convolutional layer

## History

> Originally, people created those kernels "by hand"

For example __Sobel filter__ is used to find edges in the image (read more [here](https://en.wikipedia.org/wiki/Sobel_operator))

- Those were very specific (edge detection, face detection, image gradient detection)
- Hard to come up with
- Non-specific to images

In 1989 [Yann LeCun](http://yann.lecun.com/ex/research/index.html) came up with an idea to make convolution a neural network layer to solve above shortcomings.

## Modern era

> Convolutional filter in Deep Learning has learnable connections instead of hard-fixed values

This approach solves all of the problems outlined in `History`, but we need some more nomenclature to fully understand this idea:

## Input channels

> Number of channels entering convolutional layer

![image](images/CNN_RGB.JPG)

- __Each input channel has it's own set of kernels (which are now weights)!__
- Given above we already have weights of shape `(in_channels, width, height)`

__Usually, during first convolutional layer, `in_channels=3` (Red, Green Blue) or `1` (grayscale images)__

## Output channels

> Number of channels created by convolution operation (number of input channels doesn't matter)

- Each output channel (with it's filters) convolves over __all input channels__ and sums the result
- Given above, we have weights of shape `(out_channels, in_channels, width, height)`

__Finally data is produced of shape `(batch, out_channels, width, height)`__

## Filters

> Collection of kernels (sometimes named as channels)

In case of `Conv2d` it will be `(in_channels * out_channels)`

# How this looks together

Below is a representation of:
- __SINGLE__ output channel creation
- __FROM THREE INPUT CHANNELS__
- __THOSE ARE MIXED TOGETHER__

![](./images/kernels_total.gif)

# Benefits of convolution

- Much smaller amount of parameters (when compared to `nn.Linear`)
- Same parameters go over regions of an image (with `nn.Linear` each parameter would be responsible for one pixel)
- Tailored for this specific task (architecture)
- __Much__ higher performance on spatial tasks

## What does each filter look for?

> Here's what some simple, small, 1 channel filters might look like after they've been trained.

![](images/filters.png)

> Convolutional neural networks are often represented by diagrams like the one below:

![](images/cnn.png)

# Pooling

> Pooling allows us to control when we want to shrink image's width and height

![](images/maxpoolfig.gif)

## Why?

- As the network gets depeer __we increase number of channels__ in order to learn more abstract representations, hence __computational cost increases quickly__
- Pooling allows us to control computational cost of operations
- Pooling chooses the most important features from the image

## Versions

There are a couple versions of pooling, most prevalent are:
- `MaxPooling`
- `AvgPooling` (taking average of kernel values)

### MaxPooling

- Chooses most important features
- Sharper decisions
- __Might__ be easier to train but __might__ be worse on validation
- __Most popular__
- Suitable for large networks with enough capacity (layers) to find most important features

### AvgPooling

- Takes all features into an account
- Smoother decision boundary
- __Might__ be harder to train but __might__ be better on validation
- __Less popular__
- Suitable for smaller networks with lower capacity as it doesn't leave any feature behind

## Tips

- At the start of neural network __do not use pooling__. Go for a couple layers/blocks (like 3/4) and pool after that
- After initial convolution layers you may pool every 2 layers/blocks (though you might go for more)
- Provided as [torch.nn.AvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html) 


# Global Pooling

> At the end of neural network we often need to output `(batch_size, classes)` (for classification

Due to above, we need to go from shape `(batch_size, channels, width, height)`. Given that:
- Abstract features are gathered in channels, __not width and height__

We can use so called `GlobalPooling`, provided in PyTorch as [`AdaptiveMaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveMaxPool2d.html) (specify `1` to get `GlobalPooling`):

> Global pooling works like a normal pooling, but __always return image with single pixel__, hence output shape will be `(batch_size, channels, 1, 1)` (__remember to `squeeze` dimensions before passing them to `torch.nn.Linear`!)

__Global pooling also comes in a few flavors, including `max` and `avg`__

# Putting pieces together

After this large dose of theorethical knowledge let's see a convolutional neural network in action:

## Feature creator

> __Convolutional layers create feature representation for our classifier/regressor__

General idea standing behind convolutional architectures:

- Create first convolutional layer with `in_channels` (usually `3` for `RGB`) and some `out_channels` __which is larger__ (usually `32` or multiple of it)
- Follow that by non-linear activation (usually ReLU)
- Create blocks consisting of:
    - `torch.nn.Conv2d`
    - activation
    - `torch.nn.BatchNorm2d()` - __spatial batch normalization__ (mean and variance taken separately __for each channel__)
- __Each block has predefined size (e.g. `64` channels)__
- __This block can be repeated 2/3 times__
- After that pooling layer (__usually dividing size of the image by `2`__)
- __Next block starts with larger number of channels__ (usually twice as large)
- __Process is repeated for `N` layers__ (`3` at least, usually more than `8` up to hundreds for some architectures)

Essentially:
- __Make your image smaller while SIMULTANEOUSLY increasing number of channels__
- Use similar blocks

## Classifier

> __Classifier takes `2D` input and performs our classification at the end of the network__

For modern architectures:
- Use `GlobalPooling` __to obtain only number of output channels__ (usually `512`, `1024` or so)
- Above creates a tensor of shape `(batch, out_channels)`
- Use `torch.nn.Linear(out_channels, n_classes)` to perform classification

Sometimes `Flattening` is used instead of global pooling, but:
- Imagine output size `(batch, 512, 4, 4)`
- If we flatten the above we obtain `(batch, 512 * 4 * 4)`
- This tensor goes into `nn.Linear` layer

Above approach has two major downsides:
- Size of tensor after flattening __is dependent on the size of image__
- Due to above we are limited to a single sized images (__neural networks which are not limited are called FULLY CONVOLUTIONAL NETWORKS__) because `nn.Linear` needs static `in_features`
- Usually flattening creates way too large `nn.Linear` layer __which outweighs the cost of THE WHOLE CONVOLUTIONAL PART OF THE NETWORK__ (see first architectures like `AlexNet`)

## Tips

- Give most of the parameter power for the convolutional part
- Use `GlobalPooling`
- Use skip connections (we will see it in a second)
- Base new architectures on well-known ideas (resnets, squeeze-excitation, inverted residuals in MobileNets etc.) as those were found by a large hyperaprameter search techniques unavailable to us\

## Example

Let's see an example neural network based on the above guidelines:

In [ ]:
import torch


class CNNClassifier(torch.nn.Module):
    def __init__(self, in_channels, n_classes):
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, 64, kernel_size=3),
            torch.nn.ReLU(),
            CNNClassifier._block(64),
            CNNClassifier._block(64),
            CNNClassifier._block(64),
            CNNClassifier._block(64, 128),
            # Each group of blocks shorter as it has more learnable parameters
            torch.nn.MaxPool2d(2),  # Size of image smaller by 2
            CNNClassifier._block(128),
            CNNClassifier._block(128),
            CNNClassifier._block(128, 256),
            torch.nn.MaxPool2d(2),  # Size of image smaller by 2
            # Each group of blocks shorter as it has more learnable parameters
            CNNClassifier._block(256, 256),
            CNNClassifier._block(256, 512),
            # output shape: (batch, 512, img_width, img_height)
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(1),  # output shape: (batch, 512, 1, 1)
            torch.nn.Flatten(),  # output shape: (batch, 512)
            torch.nn.Linear(512, n_classes),  # output shape: (batch, n_classes)
        )

    def forward(self, X):
        return self.classifier(self.features(X))

    @staticmethod
    def _block(in_channels, out_channels=None):
        if out_channels is None:
            out_channels = in_channels

        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(out_channels),
        )

# ResNets

__One of the most important papers in deep learning [link](https://arxiv.org/abs/1512.03385)__

> Instead of learning feature transformations as we add convolutional layers, __we learn corrections__ to the previous layers

Let's assume `F(x)` is a block containing two convolutional layers transforming `x` inputs. In ResNets case we would add a __skip connection__:

![resnet_equation](images/resnet_equation.png)

## Degradation problem

> When the layer is too deep (on the order of `100` convolutional layers) __it is too hard to optimize__

Due to that:
- __Deeper networks have larger train loss than more shallow ones!__ `50` layers would perform better than `1000` (which is not intuitive)

> __ResNets main addition is solving degradation problem__, due to that we can easily go with `1000` layer networks (though it's __almost always__ not needed for most tasks)

> Anywhere from `18` to `152` layers should be enough for any task (use the smallest one satisfying your needs


## Additional benefits

- Further (after `BatchNorm`) loss landscape smoothing
- Further reduction in vanishing/exploding gradient (though mostly taken care of by `BatchNorm` already)

## Usage tips

- Use any block and connect it using skip connection
- Usability is not limited to `convolution`, same thing applies for `linear` (though rarely for recurrent neural networks)
- __Try to keep the same size of `inputs` and `outputs` in order not to use projection__, a few resnet blocks with the same number of channels and up the number two times
- __DO NOT USE ACTIVATION AT THE END OF LAST LAYER INSIDE RESNET BLOCK__

# Exercise

Create `ResNet18` (or larger) architecture from scratch:
- Check out original research paper: https://arxiv.org/abs/1512.03385
- Look at `Figure 3` to see how the whole neural network works
- Read `Table 1` to see blocks used for different versions
- __Use either pooling or stride `2` (original research paper) to make the images smaller after the network__
- __Smoke test the architecture with random input of appropriate shape__

Additional challenges, __choose any__ (if you get through the above quickly):
- Train your architecture on `CIFAR10` via `torchvision` (or any other dataset) and using the training system we've created
- Can you generalize the `ResNet` similarly to how `torchvision` does it (e.g. different `ResNet` sizes without too much code repetition (__DRY principle__))

## Summary

- Convolutional layers are used with spatial dependent data (usually images)
- Can be used for any task just like `nn.Linear` layers
- Different versions exist (`1D`, `2D`, `3D`) for different tasks, but usually, respectively:
    - Textual (encoded) data or audio
    - Images
    - Videos (rarest case due to large amount of data)
- __Convolution can work on images of any size__ (only `channels` dimension has to be the same)
- Convolution has trainable kernels of specified size which together form filters
- We increase number of channels while simultaneuosly reducing image's size in order to learn more abstract features
- Neural network usually ends with global pooling (and optionally some task specific Linear layers, though modern architectures tend to go for single `nn.Linear`)
- Skip connections are used to combat degradation problem
- Skip connections allow us to use way deeper networks and optimize the system easier

## Challenges

- What is depthwise convolution?
- What is pointwise convolution?
- If you know above, `Separable` convolution is depthwise followed by pointwise convolution
- How does MobileNetV2 work and why is it useful?
- What is Squeeze-Excitation block, what are the upsides and downsides of adding it? Where should we add it? Read [Squeeze-Excitation research paper](https://arxiv.org/abs/1709.01507)
- Read [EfficientNet research paper ](https://arxiv.org/abs/1905.11946) to know current SOTA architecture on ImageNet
- Some additional concepts that one might want to read: shuffle nets, inception blocks, 
- How does attention on images work? Check attention on your own or __come back after attention classes__